# SOC 116AC Data Cleaning Phase 1

In [17]:
import numpy as np
import pandas as pd

import re

In [18]:
# Import data into pandas

df = pd.read_csv('survey_data.csv')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 406,Unnamed: 407,Unnamed: 408,Unnamed: 409,Unnamed: 410,Unnamed: 411,Unnamed: 412,Unnamed: 413,Unnamed: 414,Unnamed: 415
0,ID,Language,Timestamp,Organization,Do you work in a private home or in a board & ...,Do you work in a private home or in a board & ...,How many years have you been working as a dome...,How many years have you been working as a dome...,How long have you lived in the United States?,How long have you lived in the United States? ...,...,Does this employer pay you at least the minimu...,Does Employer pay you overtime pay (1.5 X your...,"Does Employer provide you paid sick time, to u...",Many workers are entitled to meal and rest bre...,Have you or someone you know ever made a compl...,Would you like an organizer or a member leader...,Would you like an organizer or a member leader...,Thank you so much for taking the time to share...,Thank you so much for taking the time to share...,10 members who have filled out the survey will...
1,1,spanish,8/31/2021 15:28:35,CHIRLA,Private homes,NaN,10 - 20 Years,NaN,10 - 20 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,Yes
2,2,english,8/31/2021 17:50:40,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,1 - 5 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,No
3,3,spanish,8/31/2021 20:27:49,CHIRLA,Other,Limpio building,6 - 10 Years,NaN,10 - 20 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,Yes
4,4,spanish,8/31/2021 20:28:04,CHIRLA,Private homes,NaN,6 - 10 Years,NaN,Other,22 años,...,NaN,NaN,NaN,NaN,NaN,Other,estoy con la organizacion CHIRLA,Yes,NaN,Yes


In [19]:
# Set columns to the right row, get rid of rows above variables 
col_names = df.iloc[0]
df.columns = col_names

df = df.drop([0])
df.head()

,ID,Language,Timestamp,Organization,Do you work in a private home or in a board & care facility with multiple adults?,Do you work in a private home or in a board & care facility with multiple adults? OTHER,How many years have you been working as a domestic worker or at a board and care home?,How many years have you been working as a domestic worker or at a board and care home? OTHER,How long have you lived in the United States?,How long have you lived in the United States? OTHER,...,"Does this employer pay you at least the minimum wage? California's minimum wage is $13 an hour, but many major cities have a higher minimum wage.",Does Employer pay you overtime pay (1.5 X your regular hourly rate) when you work shifts longer than 8 or 9 hours in a day?,"Does Employer provide you paid sick time, to use when you or your family are sick or have to go to a doctor’s appointment?",Many workers are entitled to meal and rest breaks during their shifts. Does Employer provide you meal and rest breaks when you work more than 5 hours?,Have you or someone you know ever made a complaint against Employer for their working conditions?,Would you like an organizer or a member leader from your organization give you a call to talk more about your workplace rights and to learn about the options for how to make sure those rights are respected by your employers (or also to learn how to recuperate wages from previous employers from up to three years ago?,Would you like an organizer or a member leader from your organization give you a call to talk more about your workplace rights and to learn about the options for how to make sure those rights are respected by your employers (or also to learn how to recuperate wages from previous employers from up to three years ago? OTHER,Thank you so much for taking the time to share about your experiences! We will be conducting a few longer interviews to hear more about member stories in October. Would you be willing to receive another call to do an interview then?,Thank you so much for taking the time to share about your experiences! We will be conducting a few longer interviews to hear more about member stories in October. Would you be willing to receive another call to do an interview then? OTHER,10 members who have filled out the survey will win a $25 gift card. Do you want to enter to win the raffle?
1,1,spanish,8/31/2021 15:28:35,CHIRLA,Private homes,NaN,10 - 20 Years,NaN,10 - 20 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,Yes
2,2,english,8/31/2021 17:50:40,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,1 - 5 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,No,NaN,No,NaN,No
3,3,spanish,8/31/2021 20:27:49,CHIRLA,Other,Limpio building,6 - 10 Years,NaN,10 - 20 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,Yes
4,4,spanish,8/31/2021 20:28:04,CHIRLA,Private homes,NaN,6 - 10 Years,NaN,Other,22 años,...,NaN,NaN,NaN,NaN,NaN,Other,estoy con la organizacion CHIRLA,Yes,NaN,Yes
5,5,english,9/1/2021 14:19:33,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,1 - 5 Years,NaN,...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,Yes


## Task 1: Break Down Checkbox Response Column to T/F Columns

Question: Have you experienced any of the following safety or health risks on your way to work? (please check all that apply)

Answer options:  
    None  
    Feeling unsafe commuting or walking alone  
    Overly Strenuous or physical demands  
    Lack of access to toilets, potable water, or food  
    Heat exposure  
    COVID-19 exposure  
    Wildfire smoke exposure  
    Wildfire evacuation zone  
    Harassment or Discrimination  
    Discomfort or fear at the presence of police or security officers  
    Other  


In [5]:
commute_risks_col_name = \
    'Have you experienced any of the following safety or health risks on your way to work? (please check all that apply)'

answer_options = np.array(['None',
    'Feeling unsafe commuting or walking alone',
    'Overly Strenuous or physical demands', 
    'Lack of access to toilets, potable water, or food', 
    'Heat exposure',
    'COVID-19 exposure', 
    'Wildfire smoke exposure',
    'Wildfire evacuation zone',
    'Harassment or Discrimination',
    'Discomfort or fear at the presence of police or security officers'])
# The above list does not include 'Other', because when respondents chose 'Other' and wrote a response, 
# Google Forms did not place 'Other' in their contatenated list, just their written response. 

commute_risks_col_num = np.where(col_names == commute_risks_col_name)[0][0]
commute_risks_col_num

37

### Split this concatenated responses column into seperate columns

In [6]:
# Make new columns based on if an answer was checked (True) or not (False)
col_num = int(commute_risks_col_num) + 1

for ans in answer_options:
    df.insert(col_num, ans, df[commute_risks_col_name].str.contains(ans))
    col_num += 1
    
df[answer_options].head()

,None,Feeling unsafe commuting or walking alone,Overly Strenuous or physical demands,"Lack of access to toilets, potable water, or food",Heat exposure,COVID-19 exposure,Wildfire smoke exposure,Wildfire evacuation zone,Harassment or Discrimination,Discomfort or fear at the presence of police or security officers
1,False,True,False,False,True,True,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,True,True,False,False,False,False
5,False,True,False,True,False,True,False,False,True,True


In [7]:
# Rename None column to a more useful name, like 'Never experienced health and safety risk in commute'

df = df.rename(columns={'None' : 'Never experienced health or safety risk in commute'})

df.iloc[:,32:50].head()

,Does your employer contribute to your transportation cost? DESCRIBE,Does your employer contribute to your transportation cost? OTHER,Does your employer compensate you for your time or expenses while commuting (such as gas or other fees?),Does your employer compensate you for your time or expenses while commuting (such as gas or other fees?) OTHER,"Before the Covid-19 Pandemic, what were some physical, safety, or health concerns or risks you experienced on your commute to work?",Have you experienced any of the following safety or health risks on your way to work? (please check all that apply),Never experienced health or safety risk in commute,Feeling unsafe commuting or walking alone,Overly Strenuous or physical demands,"Lack of access to toilets, potable water, or food",Heat exposure,COVID-19 exposure,Wildfire smoke exposure,Wildfire evacuation zone,Harassment or Discrimination,Discomfort or fear at the presence of police or security officers,"Are there any other costs or impacts associated with your commute? (can be emotional, monetary, physical etc.)","If you work in private homes, how many households or do you work in?"
1,NaN,NaN,No,NaN,Personas que no se asean y usan el transporte ...,"Feeling unsafe commuting or walking alone, Hea...",False,True,False,False,True,True,False,False,False,False,"Si, alguien me puede asaltar.",1-2
2,NaN,NaN,No,NaN,None,None,True,False,False,False,False,False,False,False,False,False,None,Other
3,NaN,NaN,No,NaN,Productos de limpieza,COVID-19 exposure,False,False,False,False,False,True,False,False,False,False,Monetario,Other
4,NaN,NaN,Yes,NaN,"Riesgos fisicos=caidas, resbalones,lastimadura...","Heat exposure, COVID-19 exposure",False,False,False,False,True,True,False,False,False,False,"Emocional,Emotional, miedo de contagiarme de c...",1-2
5,NaN,NaN,No,NaN,Going out from work at 11:30 midnight to go ho...,"Feeling unsafe commuting or walking alone, Lac...",False,True,False,True,False,True,False,False,True,True,Emotional,Other


### Make a seperate column for the 'Other' responses

How to get the 'Other' responses from this AH column (see row 28 on spreadsheet "I resign two months ago..." 
for 'Other' response example)

For this, I made a copy of the original column to be an 'Other' column by removing all non-Other responses from the original column, so that all that's left is the 'Other' responses. 

In [8]:
# Remove the concatenated responses that we made new columns for from the original column 
# This should only leave the 'Other' responses in our original column, and some leftover punctuation/whitespace
other_col_name = "Have you experienced any of the following safety or health risks on your way to work? (please check all that apply) OTHER"


df.insert(col_num, other_col_name, df[commute_risks_col_name])

for ans in answer_options:
    df[other_col_name] = df[other_col_name].str.replace(ans + ",", '')
    df[other_col_name] = df[other_col_name].str.replace(ans, '')
    

df[other_col_name].head(13)

1                                                      
2                                                      
3                                                      
4                                                      
5                                                      
6                                                      
7                                                      
8                                                      
9                                                      
10                                                     
11            people fighting inside the bus, standi...
12                                                 none
13                                                     
Name: Have you experienced any of the following safety or health risks on your way to work? (please check all that apply) OTHER, dtype: object

In [10]:
# Remove leading/trailing whitespace from the column
df[other_col_name] = df[other_col_name].str.strip()

In [12]:
df.iloc[:,37:52].head(13)

,Have you experienced any of the following safety or health risks on your way to work? (please check all that apply),Never experienced health or safety risk in commute,Feeling unsafe commuting or walking alone,Overly Strenuous or physical demands,"Lack of access to toilets, potable water, or food",Heat exposure,COVID-19 exposure,Wildfire smoke exposure,Wildfire evacuation zone,Harassment or Discrimination,Discomfort or fear at the presence of police or security officers,Have you experienced any of the following safety or health risks on your way to work? (please check all that apply) OTHER,"Are there any other costs or impacts associated with your commute? (can be emotional, monetary, physical etc.)","If you work in private homes, how many households or do you work in?","If you work in private homes, how many households or do you work in? OTHER"
1,"Feeling unsafe commuting or walking alone, Hea...",False,True,False,False,True,True,False,False,False,False,,"Si, alguien me puede asaltar.",1-2,NaN
2,None,True,False,False,False,False,False,False,False,False,False,,None,Other,I don't work in any private homes
3,COVID-19 exposure,False,False,False,False,False,True,False,False,False,False,,Monetario,Other,No trabajo en hogares privados
4,"Heat exposure, COVID-19 exposure",False,False,False,False,True,True,False,False,False,False,,"Emocional,Emotional, miedo de contagiarme de c...",1-2,NaN
5,"Feeling unsafe commuting or walking alone, Lac...",False,True,False,True,False,True,False,False,True,True,,Emotional,Other,I don't work in any private homes
6,Harassment or Discrimination,False,False,False,False,False,False,False,False,True,False,,Emocional,1-2,NaN
7,"Heat exposure, COVID-19 exposure, Harassment o...",False,False,False,False,True,True,False,False,True,True,,Physical,3-5,NaN
8,"Heat exposure, COVID-19 exposure, Wildfire smo...",False,False,False,False,True,True,True,False,False,False,,emotional and physical,1-2,NaN
9,"Feeling unsafe commuting or walking alone, Ove...",False,True,True,False,False,False,False,False,False,False,,None,1-2,NaN
10,None,True,False,False,False,False,False,False,False,False,False,,High cost of gas,1-2,NaN


## Task 2: Misc Cleaning -- Zip Codes, Drop Tests, Cities

### Drop Tests

Data was updated, so 'Drop Tests' section is longer needed!

In [1388]:
# This cell looks for names that are 'test' or 'yes'

# names_col_name = "First Name"

# df[df[names_col_name].str.contains(r"test|Test|^yes$", na=False)]

In [1389]:
# test_indices = df[df[names_col_name].str.contains(r"test|Test|^yes$", na=False)].index
# test_indices

In [1390]:
# df = df.drop(test_indices)

In [1391]:
# Now there's no more 'test' entries! (maybe there's other tests, this got rid of 5)

# np.count_nonzero(df[df[names_col_name].str.contains(r"test|Test|^yes$", na=False)])

### Clean Cities

In [19]:
# Make all title case (First letter of each word capitalized, rest lowercase)
# Also get rid of any extra spaces

df["City"] = df["City"].str.title()
df["City"] = df["City"].str.strip()
df["City"].head()

1    Huntington Park
2          San Diego
3        Los Angeles
4        Los Angeles
5        Los Angeles
Name: City, dtype: object

In [20]:
# Get rid of 'California' or 'Ca' in city name
df["City"] = df["City"].str.replace(r"California|Ca$|Ca\.$", '')
df["City"] = df["City"].str.strip()

In [21]:
# These are all the city entries left with more than three words 

df[df["City"].str.split().str.len() > 3]["City"]

194    (She Doesnt Know The Exact Name)
233      8329 Willis Ave, Panorama City
Name: City, dtype: object

In [22]:
# Remove character that is not either a word character or a space (punctuation)

df["City"] = df["City"].str.replace(r"[^\w\s]", '')

In [23]:
# Replace consequtive spaces with just one

df["City"] = df["City"].str.replace(r"\s{2,}", ' ')
df["City"] = df["City"].str.strip()

In [24]:
# Get rid of 'CA' after city name 

df["City"] = df["City"].str.replace(r"CA$| Cal$", '')
df["City"] = df["City"].str.strip()

In [25]:
# There's so many 'Costa Messa's

df["City"] = df["City"].str.replace(r"Costa Messa", "Costa Mesa")

# Also some just 'La's

df["City"] = df["City"].str.replace(r"^La$", "Los Angeles")
df["City"] = df["City"].str.strip()

In [26]:
# TODO: See below

print(df["City"].to_string())

1                     Huntington Park
2                           San Diego
3                         Los Angeles
4                         Los Angeles
5                         Los Angeles
6                            Petaluma
7                          Union City
8                                    
9                             Fremont
10                          San Diego
11                             Carson
12                         Montebello
13                             Carson
14                            Stanton
15                        Los Angeles
16                           Torrance
17                        Los Angeles
18                        North Hills
19                        Los Angeles
20                        Los Angeles
21                      Rolling Hills
22                              Azusa
23                       San Fernando
24                          San Diego
25                           Panorama
26                        Los Angeles
27          

In [27]:
print(df["City"].value_counts().to_string())

San Francisco                     67
Los Angeles                       61
Redwood City                      16
San Diego                         15
Santa Rosa                        15
Oakland                           14
Costa Mesa                        13
Santa Ana                         13
San Jose                          10
                                   6
三藩市                                6
Oceanside                          6
Van Nuys                           5
North Hills                        5
Sunnyvale                          5
Pomona                             5
Fremont                            5
Daly City                          4
Petaluma                           4
Anaheim                            4
National City                      4
Laguna Hills                       4
Glendale                           3
Carson                             3
San Bernardino                     3
San Mateo                          3
Pacoima                            3
P

### Clean Zipcodes

In [28]:
# Here are all the zip code entries containing non-digit characters

df[df["Zip Code"].str.contains(r"\D")]["Zip Code"]

100                92003 and 92586
101                 92026Escondido
181    94108 (said she's not sure)
275                           9???
337                        94102la
366                         94,134
Name: Zip Code, dtype: object

In [29]:
# Remove any non-digit characters 

df["Zip Code"] = df["Zip Code"].str.replace(r"\D", '')

df[df["Zip Code"].str.contains(r"\D")]["Zip Code"]

Series([], Name: Zip Code, dtype: object)

In [30]:
print(df["Zip Code"].to_string())

1           90255
2           92064
3           90027
4           91405
5           90020
6           94954
7           94587
8           94547
9           94555
10          92126
11          90745
12          90640
13          90745
14          90680
15          90028
16          90503
17          90048
18          91343
19          90004
20          90029
21          90274
22          91702
23          91343
24          92114
25          91402
26          90029
27          92630
28          90640
29          90005
30          94536
31          90049
32          94536
33          90026
34          91766
35          90501
36          90650
37          91406
38          92154
39          90028
40          90026
41          90028
42          90034
43          91766
44          91607
45          91791
46          90057
47          91402
48          90630
49          94954
50          90026
51          91343
52          90048
53          92805
54          92802
55          90631
56        

In [31]:
# Here's all of them that aren't the right length 

df[df["Zip Code"].str.len() != 5]["Zip Code"]

100    9200392586
233          9142
249          9350
275             9
316          9280
Name: Zip Code, dtype: object

In [32]:
# Cuts off all the zip codes at 5 digits

df["Zip Code"] = df["Zip Code"].str[0:5]

# Replaces all the partial zip codes with blanks

df["Zip Code"] = df["Zip Code"].str.replace(r"^\d{0,4}$", '')

In [33]:
# Now we just have missing zips, or valid zips!

df[df["Zip Code"].str.len() != 5]["Zip Code"]

233    
249    
275    
316    
Name: Zip Code, dtype: object

## Task 3: Collapse to Employer Level

The initial questions end at the question:  
When you get paid, where does the payment come from? (if you work for different types of employers, select one to start)


There are four employer types:
1. (CO) An Agency or Company
2. (DIR) I receive my pay directly from the household or family that I work for
3. (RO) Another worker or individual who manages the work, sometimes known as a route owner
4. (CF) A Residential Care Facility for the Elderly or a Board and Care Home

Their first questions are:
1. What is the name of the agency?
2. What type of work do you do for Employer 1? (please check all that apply)
3. What type of work do you do for this individual? (Check all that apply)
4. What is the name of the Board and Care Home or facility?

All four end with the question:  
Do you work for any other private household, care home, individual, or company?


For each employer type, there are 5 sets of identical questions.

In [49]:
[print(i) for i in df.columns.tolist()];

ID
Language
Timestamp
Organization
Do you work in a private home or in a board & care facility with multiple adults?
Do you work in a private home or in a board & care facility with multiple adults? OTHER
How many years have you been working as a domestic worker or at a board and care home?
How many years have you been working as a domestic worker or at a board and care home? OTHER
How long have you lived in the United States? 
How long have you lived in the United States? OTHER
Age
Gender
Gender OTHER
City
Neighborhood
Zip Code
What is the closest public park or bus or train stop to where you live?
On most days, how do you get to work?
If on public transport, do you have to make any transfers? How many?
On average, how much time do you spend traveling to work?
On average, how much time do you spend traveling to work? OTHER
If you use multiple modes of transportation, how much time do you spend in each type of transportation?
Did your mode of transportation change since the start of th

In [50]:
np.count_nonzero(df.columns == 'What type of work do you do for this individual? (Check all that apply)')

5

In [51]:
np.count_nonzero(df["What is the name of the agency?"].iloc[:,4].fillna(False))

0

In [52]:
# Here is where all of the Type 1 (Agency) questions start...

# first has gap of 22, then rest 18??
# ^^ THIS IS BECAUSE OF THE OTHER COLUMNS WE ADDED !!

# all of them length 18 (except described above, and the last 
# set of questions of each type lacks a next question, so they have 17)
# ALSO: Type 2 (direct) employers only have 17/16 questions
# Type 3 (route owner) employers have 20/19 questions?

t1 = (df.columns == "What is the name of the agency?").nonzero()[0]
t1

array([ 53,  75,  97, 115, 133])

In [53]:
t2 = (df.columns == "What type of work do you do for Employer? (please check all that apply)").nonzero()[0]
t2

array([338, 358, 375, 391, 407])

In [54]:
t3 = (df.columns == "What type of work do you do for this individual? (Check all that apply)").nonzero()[0]
t3

array([239, 259, 279, 299, 319])

In [55]:
t4 = (df.columns == "What is the name of the Board and Care Home or facility?").nonzero()[0]
t4

array([150, 168, 186, 204, 222])

In [56]:
df.iloc[:,54].count()

103

In [81]:
# This is all the data before respondents are asked individual employer questions 

df.iloc[:,:53].head()

,ID,Language,Timestamp,Organization,Do you work in a private home or in a board & care facility with multiple adults?,Do you work in a private home or in a board & care facility with multiple adults? OTHER,How many years have you been working as a domestic worker or at a board and care home?,How many years have you been working as a domestic worker or at a board and care home? OTHER,How long have you lived in the United States?,How long have you lived in the United States? OTHER,...,COVID-19 exposure,Wildfire smoke exposure,Wildfire evacuation zone,Harassment or Discrimination,Discomfort or fear at the presence of police or security officers,Have you experienced any of the following safety or health risks on your way to work? (please check all that apply) OTHER,"Are there any other costs or impacts associated with your commute? (can be emotional, monetary, physical etc.)","If you work in private homes, how many households or do you work in?","If you work in private homes, how many households or do you work in? OTHER","When you get paid, where does the payment come from? (if you work for different types of employers, select one to start)"
1,1,spanish,8/31/2021 15:28:35,CHIRLA,Private homes,NaN,10 - 20 Years,NaN,10 - 20 Years,NaN,...,True,False,False,False,False,,"Si, alguien me puede asaltar.",1-2,NaN,An Agency or Company
2,2,english,8/31/2021 17:50:40,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,1 - 5 Years,NaN,...,False,False,False,False,False,,None,Other,I don't work in any private homes,A Residential Care Facility for the Elderly or...
3,3,spanish,8/31/2021 20:27:49,CHIRLA,Other,Limpio building,6 - 10 Years,NaN,10 - 20 Years,NaN,...,True,False,False,False,False,,Monetario,Other,No trabajo en hogares privados,Another worker or individual who manages the w...
4,4,spanish,8/31/2021 20:28:04,CHIRLA,Private homes,NaN,6 - 10 Years,NaN,Other,22 años,...,True,False,False,False,False,,"Emocional,Emotional, miedo de contagiarme de c...",1-2,NaN,I receive my pay directly from the household o...
5,5,english,9/1/2021 14:19:33,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,1 - 5 Years,NaN,...,True,False,False,True,True,,Emotional,Other,I don't work in any private homes,An Agency or Company


TODO:
1. Make a generic list of questions, every employer question should be able to correspond to one of the generic questions
2. Make a dictionary that maps each employer question to a generic question
3. Write a function that'll use this dictionary to collapse data
    1. For this, make copy of dataframe. Fill the copy by iterating thru original dataframe person by person, and filling new dataframe with one row for each employer
    2. Make func. work for a row, use on each row of original dataframe

In [43]:
generic = ["asdf", "Employer Organization Name", "Types Of Work",\
 "Employer Management Name", "How Is Employee Paid", "How Often Paid",\
 "Does Employee Live In Workplace", "Number Of Clients", "Number of Other Employees",\
 "Employer City", "Employer Zip Code", "What City Meet Employer (Route Owner)",\
 "What Neighborhood Meet Employer (Route Owner)", "Employer Neighborhood", "Employer Nearest Park Or Public Transit",\
 "Does Employer Provide PPE Info And Minimize Risk", "Employer Health And Safety Precautious", "Paid At Least Minimum Wage",\
 "Overtime Pay", "Paid Sick Time", "Meal Or Rest Breaks",\
 "Complaint Against Employer"]

In [57]:
# 1-CO: company 
t1_qs = df.iloc[:,t1[2]:t1[3] - 1].columns.tolist()
t1_qs;

In [58]:
type1_dict = {t1_qs[0] : generic[1], t1_qs[1] : generic[2], t1_qs[2] : generic[3],\
              t1_qs[3] : generic[4], t1_qs[4] : generic[5], t1_qs[5] : generic[7],\
              t1_qs[6] : generic[6], t1_qs[7] : generic[9], t1_qs[8] : generic[14],\
              t1_qs[9] : generic[8], t1_qs[10] : generic[15], t1_qs[11] : generic[16],\
              t1_qs[12] : generic[17], t1_qs[13] : generic[18], t1_qs[14] : generic[19],\
              t1_qs[15] : generic[20], t1_qs[16] : generic[21]}
type1_dict;

In [59]:
# 2-DIR: direct
t2_qs = df.iloc[:,t2[3]:t2[4] - 1].columns.tolist()
t2_qs;

In [60]:
type2_dict = {t2_qs[0] : generic[2], t2_qs[1] : generic[6], t2_qs[2] : generic[9],\
              t2_qs[3] : generic[13], t2_qs[4] : generic[14], t2_qs[5] : generic[10],\
              t2_qs[6] : generic[4], t2_qs[7] : generic[5], t2_qs[8] : generic[15],\
              t2_qs[9] : generic[16], t2_qs[10] : generic[17], t2_qs[11] : generic[18],\
              t2_qs[12] : generic[19], t2_qs[13] : generic[20], t2_qs[14] : generic[21],\
              }
type2_dict;

In [61]:
# 3-RO: route owner
t3_qs = df.iloc[:,t3[2]:t3[3] - 1].columns.tolist()
t3_qs;

In [62]:
type3_dict = {t3_qs[0] : generic[2], t3_qs[1] : generic[8], t3_qs[2] : generic[7],\
              t3_qs[3] : generic[4], t3_qs[4] : generic[5], t3_qs[5] : generic[6],\
              t3_qs[6] : generic[9], t3_qs[7] : generic[13], t3_qs[8] : generic[14],\
              t3_qs[9] : generic[11], t3_qs[10] : generic[12], t3_qs[11] : generic[14],\
              t3_qs[12] : generic[15], t3_qs[13] : generic[16], t3_qs[14] : generic[17],\
              t3_qs[15] : generic[18], t3_qs[16] : generic[19], t3_qs[17] : generic[20],\
              t3_qs[18] : generic[21]
              }
type3_dict;

In [63]:
# 4-CF: care facility
t4_qs = df.iloc[:,t4[1]:t4[2] - 1].columns.tolist()
t4_qs;

In [64]:
type4_dict = {t4_qs[0] : generic[1], t4_qs[1] : generic[2], t4_qs[2] : generic[3],\
              t4_qs[3] : generic[4], t4_qs[4] : generic[5], t4_qs[5] : generic[6],\
              t4_qs[6] : generic[7], t4_qs[7] : generic[8], t4_qs[8] : generic[9],\
              t4_qs[9] : generic[14], t4_qs[10] : generic[15], t4_qs[11] : generic[16],\
              t4_qs[12] : generic[17], t4_qs[13] : generic[18], t4_qs[14] : generic[19],\
              t4_qs[15] : generic[20], t4_qs[16] : generic[21]
              }
type4_dict;

In [70]:
# Double check no missing questions

len(type4_dict) == (t4[4] - t4[3]  - 1) == 17

True

In [73]:
df[df[t1_qs[1]].iloc[:,2].notnull()][t1_qs[1]]

,What type of work do you do for this agency? (Check all that apply),What type of work do you do for this agency? (Check all that apply),What type of work do you do for this agency? (Check all that apply),What type of work do you do for this agency? (Check all that apply),What type of work do you do for this agency? (Check all that apply)
43,NaN,Staffing/Hiring,Homecare for an older adult or person with dis...,NaN,NaN


In [71]:
# This formula gets a table of just one employee response! (the Nth response)
# Change N from 0 to 4
N = 1

df[t1_qs].iloc[:,N::5].head(10)

,What is the name of the agency?,What type of work do you do for this agency? (Check all that apply),What is the name of the owner (s) of the agency?,How does the agency pay you?,How often does the agency pay you?,How many households or clients do you work for through this agency?,Do you live in the home of one of the households you work for through this agency?,What city is the agency located?,What is the agency's address or the nearest public park or bus or train stop?,How many workers work for this agency? (What is your best guess),"Since the onset of COVID-19, does the agency provide PPE like gloves, masks, or sanitizer, share information with you and other workers about exposure, and minimize the risk of through social distance guidelines, ventilation, or training?",Does the agency take any of the following general health and safety precautions? (check all that apply),"Does this agency pay you at least the minimum wage? California's minimum wage is $13 an hour, but many major cities have a higher minimum wage.",Does this agency pay you overtime pay (1.5 X your regular hourly rate) when you work shifts longer than 8 or 9 hours in a day?,"Does this agency provide you paid sick time, to use when you or your family are sick or have to go to a doctor’s appointment?",Many workers are entitled to meal and rest breaks during their shifts. Does this agency provide you meal and rest breaks when you work more than 5 hours?,Have you or someone you know ever made a complaint against this agency for their working conditions?
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Caringtouch inhome care,Caregiver at a Board and Care Home,Mr. torralba,Per hour,Twice a month,1-2,No,Carlsbad,Don’t know,10-19,Yes,Provide information on the use of chemical cle...,Yes,I never work that many hours for this employer...,No,No,No
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# Here's our function to squash the data

df_flat = df.iloc[0:0, 0:53] # This is all the data before the employer responses (personal data)

# Add our generic employer question columns, and employer ID and type columns
df_flat_existing_cols = df_flat.columns.tolist()
df_flat = df_flat.reindex(columns = [df_flat_existing_cols[0]] +\
                          ['Employer ID','Employer Type'] + df_flat_existing_cols[1:] + generic)

dicts = [type1_dict, type2_dict, type3_dict, type4_dict]

# Iterate thru rows
for r in df.iterrows():
    datarow = r[1]
    employer_id = 0
    employer_type = 0
    # Iterate thru types of employers
    for employer_qs in [t1_qs, t2_qs, t3_qs, t4_qs]:
        employer_type += 1
        transl = dicts[employer_type - 1]
        # Iterate thru employer entries
        for N in range(0, 5):
            # Blank entry
            if datarow[employer_qs[0]].isnull().iloc[N]:
                continue
            # Filled entry, to process!
            else:
                employer_id += 1
                # Make row with consistent employee data, employer ID, employer type
                df_flat = df_flat.append(datarow[0:53], ignore_index=True)
                df_flat.loc[df_flat.index[-1], ["Employer ID"]] = employer_id
                df_flat.loc[df_flat.index[-1], ["Employer Type"]] = employer_type
                #datarow[employer_qs][N::5] is all responses we care about
                # Iterate thru questions in a response
                for q in employer_qs:
                    newentry = datarow[employer_qs][N::5][q]
                    if type(newentry) == pd.core.series.Series:
                        # Double check if this doesn't break any data 
                        newentry = datarow[employer_qs][N::5][q].iloc[0]
                    df_flat.loc[df_flat.index[-1], [transl[q]]] = newentry
                    
                
df_flat["Employer ID"] = df_flat["Employer ID"].astype(int)     
df_flat["Employer Type"] = df_flat["Employer Type"].astype(int)     

In [84]:
df_flat.head()

,ID,Employer ID,Employer Type,Language,Timestamp,Organization,Do you work in a private home or in a board & care facility with multiple adults?,Do you work in a private home or in a board & care facility with multiple adults? OTHER,How many years have you been working as a domestic worker or at a board and care home?,How many years have you been working as a domestic worker or at a board and care home? OTHER,...,What Neighborhood Meet Employer (Route Owner),Employer Neighborhood,Employer Nearest Park Or Public Transit,Does Employer Provide PPE Info And Minimize Risk,Employer Health And Safety Precautious,Paid At Least Minimum Wage,Overtime Pay,Paid Sick Time,Meal Or Rest Breaks,Complaint Against Employer
0,1,1,1,spanish,8/31/2021 15:28:35,CHIRLA,Private homes,NaN,10 - 20 Years,NaN,...,NaN,NaN,Randolph y pacific,Yes,"Provide general PPE like gloves or masks etc.,...",Yes,Yes,No,Yes,I don't know
1,2,1,1,english,8/31/2021 17:50:40,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,...,NaN,NaN,Don’t know,Yes,Provide information on the use of chemical cle...,Yes,I never work that many hours for this employer...,No,No,No
2,2,2,4,english,8/31/2021 17:50:40,Pilipino Workers Center,Board and Care Home,NaN,1 - 5 Years,NaN,...,NaN,NaN,Poway park,Yes,Provide information on the use of chemical cle...,No,I never work that many hours for this employer...,No,Yes,No
3,3,1,3,spanish,8/31/2021 20:27:49,CHIRLA,Other,Limpio building,6 - 10 Years,NaN,...,Los angeles,No se,De pende lugar a ir,Sí,Proporcione equipo protectivo personal general...,Sí,Nunca trabajo tantas horas para este empleador...,No lo sé,Sí,No
4,4,1,2,spanish,8/31/2021 20:28:04,CHIRLA,Private homes,NaN,6 - 10 Years,NaN,...,NaN,NaN,Montana y Sepulveda,Yes,Provide information on the use of chemical cle...,Yes,Yes,Yes,Yes,No


In [1378]:
# Now let's export df_flat !!!!

df_flat.to_excel("output.xlsx")